In [67]:
import tensorflow as tf
from collections import Counter

train_file = open("./simple-examples/data/ptb.train.txt", 'r')
train_data = train_file.read().replace("\n", "<eos>").split()

c = Counter(train_data)
count_pairs = sorted(c.items(), key=lambda x: (-x[1], x[0]))
words, _ = zip(*count_pairs)
word_to_id = dict(zip(words, range(len(words))))
word_to_id = [word_to_id[word] for word in train_data if word in word_to_id]

In [68]:
init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 5
num_layers = 2
num_steps = 20
hidden_size = 200
max_epoch = 4
max_max_epoch = 13
keep_prob = 1.0
lr_decay = 0.5
batch_size = 20
vocab_size = 10000

In [64]:
with tf.Graph().as_default():
    raw_data = tf.convert_to_tensor(word_to_id, name="raw_data", dtype=tf.int32)
    data_len = tf.size(raw_data)
    batch_len = data_len // batch_size
    data = tf.reshape(raw_data[0 : batch_size * batch_len],[batch_size, batch_len])
    epoch_size = (batch_len - 1) // num_steps
    epoch_size = tf.identity(epoch_size, name="epoch_size")

    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = tf.strided_slice(data, [0, i * num_steps], [batch_size, (i + 1) * num_steps])
    x.set_shape([batch_size, num_steps])
    y = tf.strided_slice(data, [0, i * num_steps + 1], [batch_size, (i + 1) * num_steps + 1])
    y.set_shape([batch_size, num_steps])
    
    
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0, state_is_tuple=True)
    lstm_cell_with_dropout = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell_with_dropout for _ in range(num_layers)], state_is_tuple=True)
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    embedding = tf.Variable([vocab_size, hidden_size], dtype=tf.float32)
    inputs = tf.nn.embedding_lookup(embedding, x)
    inputs = tf.nn.dropout(inputs, keep_prob)

    output = list()
    state = initial_state
    for time_step in range(num_steps):
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)

#     output = tf.reshape(tf.stack(axis=1, values=outputs), [-1, hidden_size])
#     softmax_w = tf.Variable(tf.random_normal([hidden_size, vocab_size]))
#     softmax_b = tf.Variable(tf.random_normal([vocab_size]))
#     logits = tf.matmul(output, softmax_w) + softmax_b

#     # Reshape logits to be 3-D tensor for sequence loss
#     logits = tf.reshape(logits, [batch_size, num_steps, 25])

    # use the contrib sequence loss and average over the batches
    # loss = tf.contrib.seq2seq.sequence_loss(
    #     logits,
    #     y,
    #     tf.ones([batch_size, num_steps], dtype=tf.float32),
    #     average_across_timesteps=False,
    #     average_across_batch=True
    # )

    # update the cost variables
    # cost = tf.reduce_sum(loss)

    # lr = tf.Variable(0.0, trainable=False)
    # tvars = tf.trainable_variables()
    # grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
    # optimizer = tf.train.GradientDescentOptimizer(lr)
    # train_op = optimizer.apply_gradients(zip(grads, tvars), global_step=tf.contrib.framework.get_or_create_global_step())

    # new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
    # lr_update = tf.assign(lr, new_lr)

Tensor("Size:0", shape=(), dtype=int32)


ValueError: Index out of range using input dim 2; input has only 2 dims for 'strided_slice_1' (op: 'StridedSlice') with input shapes: [20,20], [3], [3], [3] and with computed input tensors: input[3] = <1 1 1>.